In [1]:
#pip install pyspark

In [2]:
# mounting google colab

In [3]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)


Mounted at /content/gdrive


In [4]:
cd /content/gdrive/MyDrive/pyspark/kaggle

/content/gdrive/MyDrive/pyspark/kaggle


In [5]:
# pip install kaggle

In [6]:
cp /root/kaggle/kaggle.json  /root/.kaggle

In [7]:
# ! kaggle datasets download usdot/flight-delays

In [8]:
# ! unzip /content/gdrive/MyDrive/pyspark/kaggle/flight-delays.zip

In [9]:
# importing library

In [10]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import time

from pyspark.sql.functions import monotonically_increasing_id,broadcast

In [11]:
#creating spark session
my_spark = SparkSession.builder.getOrCreate()
my_spark

In [12]:
#importing files
flights = my_spark.read.csv("/content/gdrive/MyDrive/pyspark/kaggle/flights.csv",header=True)
airports = my_spark.read.csv("/content/gdrive/MyDrive/pyspark/kaggle/airports.csv",header=True)
airlines = my_spark.read.csv("/content/gdrive/MyDrive/pyspark/kaggle/airlines.csv",header=True)

In [13]:
# printing the schema

In [14]:
flights.printSchema()

root
 |-- YEAR: string (nullable = true)
 |-- MONTH: string (nullable = true)
 |-- DAY: string (nullable = true)
 |-- DAY_OF_WEEK: string (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: string (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: string (nullable = true)
 |-- DEPARTURE_TIME: string (nullable = true)
 |-- DEPARTURE_DELAY: string (nullable = true)
 |-- TAXI_OUT: string (nullable = true)
 |-- WHEELS_OFF: string (nullable = true)
 |-- SCHEDULED_TIME: string (nullable = true)
 |-- ELAPSED_TIME: string (nullable = true)
 |-- AIR_TIME: string (nullable = true)
 |-- DISTANCE: string (nullable = true)
 |-- WHEELS_ON: string (nullable = true)
 |-- TAXI_IN: string (nullable = true)
 |-- SCHEDULED_ARRIVAL: string (nullable = true)
 |-- ARRIVAL_TIME: string (nullable = true)
 |-- ARRIVAL_DELAY: string (nullable = true)
 |-- D

In [15]:
# checking class 
type(flights)

pyspark.sql.dataframe.DataFrame

In [16]:
# checking Distance Data Type
flights.select('Distance').dtypes

[('Distance', 'string')]

In [17]:
# casting string to integer
flights = flights.withColumn("Distance",flights['Distance'].cast('integer'))
flights.select('Distance').dtypes

[('Distance', 'int')]

In [18]:
# top 5 
flights.show(5)

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|Distance|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

In [19]:
airports.show(5)

+---------+--------------------+-----------+-----+-------+--------+----------+
|IATA_CODE|             AIRPORT|       CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+---------+--------------------+-----------+-----+-------+--------+----------+
|      ABE|Lehigh Valley Int...|  Allentown|   PA|    USA|40.65236| -75.44040|
|      ABI|Abilene Regional ...|    Abilene|   TX|    USA|32.41132| -99.68190|
|      ABQ|Albuquerque Inter...|Albuquerque|   NM|    USA|35.04022|-106.60919|
|      ABR|Aberdeen Regional...|   Aberdeen|   SD|    USA|45.44906| -98.42183|
|      ABY|Southwest Georgia...|     Albany|   GA|    USA|31.53552| -84.19447|
+---------+--------------------+-----------+-----+-------+--------+----------+
only showing top 5 rows



In [20]:
airlines.show(5)

+---------+--------------------+
|IATA_CODE|             AIRLINE|
+---------+--------------------+
|       UA|United Air Lines ...|
|       AA|American Airlines...|
|       US|     US Airways Inc.|
|       F9|Frontier Airlines...|
|       B6|     JetBlue Airways|
+---------+--------------------+
only showing top 5 rows



In [21]:
# creating column duration_hrs
flights= flights.withColumn('duration_hrs',flights.AIR_TIME/60)
flights.show(5)

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+-----------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|Distance|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|     duration_hrs|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--

In [22]:
# Filter

In [23]:
# filter Distance > 1000
dist_flights = flights.filter('DISTANCE>1000')
dist_flights.show(1)

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+-----------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|Distance|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|     duration_hrs|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--

In [24]:
# selecting specific columns
dist_col = dist_flights.select('YEAR','MONTH','FLIGHT_NUMBER','TAIL_NUMBER','ORIGIN_AIRPORT','DESTINATION_AIRPORT','AIR_TIME','DISTANCE')
dist_col.show(5)

+----+-----+-------------+-----------+--------------+-------------------+--------+--------+
|YEAR|MONTH|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|AIR_TIME|DISTANCE|
+----+-----+-------------+-----------+--------------+-------------------+--------+--------+
|2015|    1|           98|     N407AS|           ANC|                SEA|     169|    1448|
|2015|    1|         2336|     N3KUAA|           LAX|                PBI|     263|    2330|
|2015|    1|          840|     N171US|           SFO|                CLT|     266|    2296|
|2015|    1|          258|     N3HYAA|           LAX|                MIA|     258|    2342|
|2015|    1|          135|     N527AS|           SEA|                ANC|     199|    1448|
+----+-----+-------------+-----------+--------------+-------------------+--------+--------+
only showing top 5 rows



In [25]:
# Filtering Column based where Destination Airport == PBI
dist_col.filter(dist_col.DESTINATION_AIRPORT=='PBI').show(5)

+----+-----+-------------+-----------+--------------+-------------------+--------+--------+
|YEAR|MONTH|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|AIR_TIME|DISTANCE|
+----+-----+-------------+-----------+--------------+-------------------+--------+--------+
|2015|    1|         2336|     N3KUAA|           LAX|                PBI|     263|    2330|
|2015|    1|          721|     N623JB|           BOS|                PBI|     170|    1197|
|2015|    1|          353|     N570JB|           JFK|                PBI|     142|    1028|
|2015|    1|         1467|     N216JB|           HPN|                PBI|     146|    1056|
|2015|    1|          453|     N652JB|           JFK|                PBI|     143|    1028|
+----+-----+-------------+-----------+--------------+-------------------+--------+--------+
only showing top 5 rows



In [26]:
# filter based on Multiple Columns
dist_col.filter(dist_col.ORIGIN_AIRPORT=='JFK').filter(dist_col.DESTINATION_AIRPORT=='PBI').show(5)

+----+-----+-------------+-----------+--------------+-------------------+--------+--------+
|YEAR|MONTH|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|AIR_TIME|DISTANCE|
+----+-----+-------------+-----------+--------------+-------------------+--------+--------+
|2015|    1|          353|     N570JB|           JFK|                PBI|     142|    1028|
|2015|    1|          453|     N652JB|           JFK|                PBI|     143|    1028|
|2015|    1|         2012|     N954DL|           JFK|                PBI|     147|    1028|
|2015|    1|          653|     N554JB|           JFK|                PBI|     144|    1028|
|2015|    1|           53|     N534JB|           JFK|                PBI|     141|    1028|
+----+-----+-------------+-----------+--------------+-------------------+--------+--------+
only showing top 5 rows



select() method is useful when you simply need to select a subset of columns from a particular Spark DataFrame. On the other hand, selectExpr() comes in handy when you need to select particular columns while at the same time you also need to apply some sort of transformation over particular column(s).



In [27]:
# selecting columns based on Select Expr
dist_col.selectExpr('YEAR','MONTH','FLIGHT_NUMBER','TAIL_NUMBER','ORIGIN_AIRPORT','DESTINATION_AIRPORT','AIR_TIME','DISTANCE','DISTANCE/(AIR_TIME / 60)AS Average_Speed').show(5)

+----+-----+-------------+-----------+--------------+-------------------+--------+--------+-----------------+
|YEAR|MONTH|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|AIR_TIME|DISTANCE|    Average_Speed|
+----+-----+-------------+-----------+--------------+-------------------+--------+--------+-----------------+
|2015|    1|           98|     N407AS|           ANC|                SEA|     169|    1448|514.0828402366864|
|2015|    1|         2336|     N3KUAA|           LAX|                PBI|     263|    2330|531.5589353612166|
|2015|    1|          840|     N171US|           SFO|                CLT|     266|    2296|517.8947368421052|
|2015|    1|          258|     N3HYAA|           LAX|                MIA|     258|    2342|544.6511627906978|
|2015|    1|          135|     N527AS|           SEA|                ANC|     199|    1448|436.5829145728643|
+----+-----+-------------+-----------+--------------+-------------------+--------+--------+-----------------+
only showi

In [28]:
dist_col.count()

1646627

In [29]:
# grouping columns based on Origin Airport = USA and doing count
dist_col.filter(dist_col.ORIGIN_AIRPORT=='SEA').groupby('ORIGIN_AIRPORT').count().show()

+--------------+-----+
|ORIGIN_AIRPORT|count|
+--------------+-----+
|           SEA|59873|
+--------------+-----+



In [30]:
dist_col = dist_col.withColumn('AIR_TIME',dist_col['AIR_TIME'].cast('integer'))
dist_col

DataFrame[YEAR: string, MONTH: string, FLIGHT_NUMBER: string, TAIL_NUMBER: string, ORIGIN_AIRPORT: string, DESTINATION_AIRPORT: string, AIR_TIME: int, DISTANCE: int]

Aggregate Columns with :-
Minimum
Maximum
Average
Sum

In [31]:
dist_col.select('DISTANCE').groupby().max().show()

+-------------+
|max(DISTANCE)|
+-------------+
|         4983|
+-------------+



In [32]:
dist_col.select('DISTANCE').groupby().min().show()

+-------------+
|min(DISTANCE)|
+-------------+
|         1004|
+-------------+



In [33]:
dist_col.select('DISTANCE').groupby().avg().show()

+------------------+
|     avg(DISTANCE)|
+------------------+
|1605.6454473296017|
+------------------+



In [34]:
dist_col.groupby().sum('DISTANCE').collect()[0][0]

2643899146

In [35]:
# filter where Origin Airport = SEA grouping by Destination Airport and Count
dist_col.filter(dist_col.ORIGIN_AIRPORT=='SEA').groupby('DESTINATION_AIRPORT').count().show(5)

+-------------------+-----+
|DESTINATION_AIRPORT|count|
+-------------------+-----+
|                MSY|  334|
|                DCA|  657|
|                LIH|  562|
|                IAH| 2204|
|                HNL| 1744|
+-------------------+-----+
only showing top 5 rows



In [36]:
month_df = dist_col.groupby('MONTH','ORIGIN_AIRPORT')

In [37]:
month_df.avg('DISTANCE').show(5)

+-----+--------------+------------------+
|MONTH|ORIGIN_AIRPORT|     avg(DISTANCE)|
+-----+--------------+------------------+
|    1|           BNA|1423.3386773547095|
|    2|           LIT|            1176.0|
|    3|           BDL| 1243.851485148515|
|    1|           PDX|1759.3812654067378|
|    1|           BUR|            2465.0|
+-----+--------------+------------------+
only showing top 5 rows



In [38]:
# aggregate function
month_df.agg(F.mean('DISTANCE')).show(5)

+-----+--------------+------------------+
|MONTH|ORIGIN_AIRPORT|     avg(DISTANCE)|
+-----+--------------+------------------+
|    1|           BNA|1423.3386773547095|
|    2|           LIT|            1176.0|
|    3|           BDL| 1243.851485148515|
|    1|           PDX|1759.3812654067378|
|    1|           BUR|            2465.0|
+-----+--------------+------------------+
only showing top 5 rows



In [39]:
# checking number of partitions for the dataframe
flights.rdd.getNumPartitions()

5

In [40]:
#filtering distinct columns
airports.select(airports['AIRPORT']).distinct().show(5)

+--------------------+
|             AIRPORT|
+--------------------+
|Melbourne Interna...|
|     Eppley Airfield|
|San Diego Interna...|
|     Kahului Airport|
|Austin-Bergstrom ...|
+--------------------+
only showing top 5 rows



In [41]:
# filtering based on length
airports.filter('length(AIRPORT)<15').show()

+---------+--------------+--------------------+-----+-------+--------+----------+
|IATA_CODE|       AIRPORT|                CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+---------+--------------+--------------------+-----+-------+--------+----------+
|      ACV|Arcata Airport|       Arcata/Eureka|   CA|    USA|40.97812|-124.10862|
|      ADK|  Adak Airport|                Adak|   AK|    USA|51.87796|-176.64603|
|      ADQ|Kodiak Airport|              Kodiak|   AK|    USA|57.74997|-152.49386|
|      BET|Bethel Airport|              Bethel|   AK|    USA|60.77978|-161.83800|
|      BFL| Meadows Field|         Bakersfield|   CA|    USA|35.43360|-119.05677|
|      CRW|Yeager Airport|          Charleston|   WV|    USA|38.37315| -81.59319|
|      IMT|  Ford Airport|Iron Mountain/Kin...|   MI|    USA|45.81835| -88.11454|
|      LIH| Lihue Airport|               Lihue|   HI|    USA|21.97598|-159.33896|
|      OME|  Nome Airport|                Nome|   AK|    USA|64.51220|-165.44525|
|      VEL|Valde

In [42]:
# filtering such that Airport column will not contain
airports.filter(~F.column('AIRPORT').contains('AIRPORT')).show()

+---------+--------------------+-------------+-----+-------+--------+----------+
|IATA_CODE|             AIRPORT|         CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+---------+--------------------+-------------+-----+-------+--------+----------+
|      ABE|Lehigh Valley Int...|    Allentown|   PA|    USA|40.65236| -75.44040|
|      ABI|Abilene Regional ...|      Abilene|   TX|    USA|32.41132| -99.68190|
|      ABQ|Albuquerque Inter...|  Albuquerque|   NM|    USA|35.04022|-106.60919|
|      ABR|Aberdeen Regional...|     Aberdeen|   SD|    USA|45.44906| -98.42183|
|      ABY|Southwest Georgia...|       Albany|   GA|    USA|31.53552| -84.19447|
|      ACK|Nantucket Memoria...|    Nantucket|   MA|    USA|41.25305| -70.06018|
|      ACT|Waco Regional Air...|         Waco|   TX|    USA|31.61129| -97.23052|
|      ACV|      Arcata Airport|Arcata/Eureka|   CA|    USA|40.97812|-124.10862|
|      ACY|Atlantic City Int...|Atlantic City|   NJ|    USA|39.45758| -74.57717|
|      ADK|        Adak Airp

Conditional Statement Execution

In [43]:
airports.withColumn('State Name',F.when(airports.STATE=='TX','TEXAS')).show(5)

+---------+--------------------+-----------+-----+-------+--------+----------+----------+
|IATA_CODE|             AIRPORT|       CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|State Name|
+---------+--------------------+-----------+-----+-------+--------+----------+----------+
|      ABE|Lehigh Valley Int...|  Allentown|   PA|    USA|40.65236| -75.44040|      null|
|      ABI|Abilene Regional ...|    Abilene|   TX|    USA|32.41132| -99.68190|     TEXAS|
|      ABQ|Albuquerque Inter...|Albuquerque|   NM|    USA|35.04022|-106.60919|      null|
|      ABR|Aberdeen Regional...|   Aberdeen|   SD|    USA|45.44906| -98.42183|      null|
|      ABY|Southwest Georgia...|     Albany|   GA|    USA|31.53552| -84.19447|      null|
+---------+--------------------+-----------+-----+-------+--------+----------+----------+
only showing top 5 rows



In [44]:
airports.withColumn('Flag',F.when(airports.STATE == 'TX','TEXAS').when(airports.STATE=='GA','Georgio').otherwise('N/A')).show(5)

+---------+--------------------+-----------+-----+-------+--------+----------+-------+
|IATA_CODE|             AIRPORT|       CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|   Flag|
+---------+--------------------+-----------+-----+-------+--------+----------+-------+
|      ABE|Lehigh Valley Int...|  Allentown|   PA|    USA|40.65236| -75.44040|    N/A|
|      ABI|Abilene Regional ...|    Abilene|   TX|    USA|32.41132| -99.68190|  TEXAS|
|      ABQ|Albuquerque Inter...|Albuquerque|   NM|    USA|35.04022|-106.60919|    N/A|
|      ABR|Aberdeen Regional...|   Aberdeen|   SD|    USA|45.44906| -98.42183|    N/A|
|      ABY|Southwest Georgia...|     Albany|   GA|    USA|31.53552| -84.19447|Georgio|
+---------+--------------------+-----------+-----+-------+--------+----------+-------+
only showing top 5 rows



In [45]:
#monotonically increasing id
airports.withColumn('ID',monotonically_increasing_id()).show()

+---------+--------------------+-------------+-----+-------+--------+----------+---+
|IATA_CODE|             AIRPORT|         CITY|STATE|COUNTRY|LATITUDE| LONGITUDE| ID|
+---------+--------------------+-------------+-----+-------+--------+----------+---+
|      ABE|Lehigh Valley Int...|    Allentown|   PA|    USA|40.65236| -75.44040|  0|
|      ABI|Abilene Regional ...|      Abilene|   TX|    USA|32.41132| -99.68190|  1|
|      ABQ|Albuquerque Inter...|  Albuquerque|   NM|    USA|35.04022|-106.60919|  2|
|      ABR|Aberdeen Regional...|     Aberdeen|   SD|    USA|45.44906| -98.42183|  3|
|      ABY|Southwest Georgia...|       Albany|   GA|    USA|31.53552| -84.19447|  4|
|      ACK|Nantucket Memoria...|    Nantucket|   MA|    USA|41.25305| -70.06018|  5|
|      ACT|Waco Regional Air...|         Waco|   TX|    USA|31.61129| -97.23052|  6|
|      ACV|      Arcata Airport|Arcata/Eureka|   CA|    USA|40.97812|-124.10862|  7|
|      ACY|Atlantic City Int...|Atlantic City|   NJ|    USA|39.45

In [46]:
# cache in pyspark
start_time = time.time()

dest_cache = flights.select('Destination_AIRPORT').cache()
print('First Call to cache ',dest_cache, time.time()- start_time)
second_time = time.time()
print('Second Call to the Dataframe',dest_cache,time.time()-second_time)

First Call to cache  DataFrame[Destination_AIRPORT: string] 0.0643320083618164
Second Call to the Dataframe DataFrame[Destination_AIRPORT: string] 7.987022399902344e-05


In [47]:
# clearing cache
print('Is the dataframe Cached?',dest_cache.is_cached)
dest_cache.unpersist()
print('Is the dataframe Cached?',dest_cache.is_cached)

Is the dataframe Cached? True
Is the dataframe Cached? False


Joining Dataframes in Pyspark

In [48]:
df= airports.join(flights,airports['IATA_CODE']==flights['ORIGIN_AIRPORT'])

Explain :- Prints the (logical and physical) plans to the console for debugging purpose.


In [49]:
df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [IATA_CODE#96], [ORIGIN_AIRPORT#24], Inner, BuildLeft, false
   :- BroadcastExchange HashedRelationBroadcastMode(List(input[0, string, false]),false), [id=#783]
   :  +- Filter isnotnull(IATA_CODE#96)
   :     +- FileScan csv [IATA_CODE#96,AIRPORT#97,CITY#98,STATE#99,COUNTRY#100,LATITUDE#101,LONGITUDE#102] Batched: false, DataFilters: [isnotnull(IATA_CODE#96)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/content/gdrive/MyDrive/pyspark/kaggle/airports.csv], PartitionFilters: [], PushedFilters: [IsNotNull(IATA_CODE)], ReadSchema: struct<IATA_CODE:string,AIRPORT:string,CITY:string,STATE:string,COUNTRY:string,LATITUDE:string,LO...
   +- Project [YEAR#17, MONTH#18, DAY#19, DAY_OF_WEEK#20, AIRLINE#21, FLIGHT_NUMBER#22, TAIL_NUMBER#23, ORIGIN_AIRPORT#24, DESTINATION_AIRPORT#25, SCHEDULED_DEPARTURE#26, DEPARTURE_TIME#27, DEPARTURE_DELAY#28, TAXI_OUT#29, WHEELS_OFF#30, SCHEDULED_TIME#31, ELAPSED_TIME#32, AI

In [50]:
df_broadcast = airports.join(broadcast(flights),airports['IATA_CODE']==flights['ORIGIN_AIRPORT'])

In [51]:
df_broadcast.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [IATA_CODE#96], [ORIGIN_AIRPORT#24], Inner, BuildRight, false
   :- Filter isnotnull(IATA_CODE#96)
   :  +- FileScan csv [IATA_CODE#96,AIRPORT#97,CITY#98,STATE#99,COUNTRY#100,LATITUDE#101,LONGITUDE#102] Batched: false, DataFilters: [isnotnull(IATA_CODE#96)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/content/gdrive/MyDrive/pyspark/kaggle/airports.csv], PartitionFilters: [], PushedFilters: [IsNotNull(IATA_CODE)], ReadSchema: struct<IATA_CODE:string,AIRPORT:string,CITY:string,STATE:string,COUNTRY:string,LATITUDE:string,LO...
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[7, string, true]),false), [id=#806]
      +- Project [YEAR#17, MONTH#18, DAY#19, DAY_OF_WEEK#20, AIRLINE#21, FLIGHT_NUMBER#22, TAIL_NUMBER#23, ORIGIN_AIRPORT#24, DESTINATION_AIRPORT#25, SCHEDULED_DEPARTURE#26, DEPARTURE_TIME#27, DEPARTURE_DELAY#28, TAXI_OUT#29, WHEELS_OFF#30, SCHEDULED_TIME#31, ELAPSED_TIME#32, AIR_T

creating Temporary Table in Pyspark

In [52]:
airports.createOrReplaceTempView('Airports_tbl')
my_spark.sql('Select * from Airports_tbl').show()

+---------+--------------------+-------------+-----+-------+--------+----------+
|IATA_CODE|             AIRPORT|         CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+---------+--------------------+-------------+-----+-------+--------+----------+
|      ABE|Lehigh Valley Int...|    Allentown|   PA|    USA|40.65236| -75.44040|
|      ABI|Abilene Regional ...|      Abilene|   TX|    USA|32.41132| -99.68190|
|      ABQ|Albuquerque Inter...|  Albuquerque|   NM|    USA|35.04022|-106.60919|
|      ABR|Aberdeen Regional...|     Aberdeen|   SD|    USA|45.44906| -98.42183|
|      ABY|Southwest Georgia...|       Albany|   GA|    USA|31.53552| -84.19447|
|      ACK|Nantucket Memoria...|    Nantucket|   MA|    USA|41.25305| -70.06018|
|      ACT|Waco Regional Air...|         Waco|   TX|    USA|31.61129| -97.23052|
|      ACV|      Arcata Airport|Arcata/Eureka|   CA|    USA|40.97812|-124.10862|
|      ACY|Atlantic City Int...|Atlantic City|   NJ|    USA|39.45758| -74.57717|
|      ADK|        Adak Airp

In [53]:
my_spark.sql('select * from airports_tbl where state ="PA"').show()

+---------+--------------------+--------------------+-----+-------+--------+---------+
|IATA_CODE|             AIRPORT|                CITY|STATE|COUNTRY|LATITUDE|LONGITUDE|
+---------+--------------------+--------------------+-----+-------+--------+---------+
|      ABE|Lehigh Valley Int...|           Allentown|   PA|    USA|40.65236|-75.44040|
|      AVP|Wilkes-Barre/Scra...|Wilkes-Barre/Scra...|   PA|    USA|41.33815|-75.72427|
|      ERI|Erie Internationa...|                Erie|   PA|    USA|42.08202|-80.17622|
|      LBE|Arnold Palmer Reg...|             Latrobe|   PA|    USA|40.27594|-79.40480|
|      MDT|Harrisburg Intern...|          Harrisburg|   PA|    USA|40.19350|-76.76340|
|      PHL|Philadelphia Inte...|        Philadelphia|   PA|    USA|39.87195|-75.24114|
|      PIT|Pittsburgh Intern...|          Pittsburgh|   PA|    USA|40.49147|-80.23287|
|      SCE|University Park A...|       State College|   PA|    USA|40.85121|-77.84630|
+---------+--------------------+-----------